In [1]:
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_pickle('treino.pkl.gz')

# IDADE

In [2]:
df['nascimento'].isnull().value_counts()

False    186523
Name: nascimento, dtype: int64

In [3]:
# possíveis anos de nascimento
# suponho que todos sejam '19--', com exceção de '1900', neste caso, considero que o ano seja '2000'
df['nascimento'].str[6:].unique()

array(['76', '97', '83', '94', '92', '80', '79', '95', '96', '72', '73',
       '71', '88', '84', '61', '70', '75', '85', '69', '77', '67', '82',
       '74', '89', '63', '64', '87', '65', '91', '81', '90', '78', '86',
       '93', '99', '68', '62', '98', '55', '58', '59', '66', '00', '60',
       '56', '54', '57', '49'], dtype=object)

In [4]:
def year_to_age(x):
    if x == '00':
        return 20
    else:
        return (100 - int(x)) + 20

df['nascimento'] = df['nascimento'].str[6:].apply(lambda x: year_to_age(x))
df = df.rename(columns = {'nascimento': 'idade'})

# DATA CONTRATO

In [5]:
df['data_contrato'].isnull().value_counts()

False    186523
Name: data_contrato, dtype: int64

In [6]:
df['data_contrato'].str[6:].unique()

# todos os contratos são de 2018

array(['18'], dtype=object)

In [7]:
df['data_contrato'].str[3:5].unique()

# dentro do ano 2018, os possíveis meses são agosto, setembro e outubro
# dividirei os dados entre estes meses

array(['09', '10', '08'], dtype=object)

In [8]:
def month_split(x):
    if x == '08':
        return 'agosto'
    elif x == '09':
        return 'setembro'
    else: 
        return 'outubro'
    
df['data_contrato'] = df['data_contrato'].str[3:5].apply(lambda x: month_split(x))
df = df.rename(columns = {'data_contrato': 'mes_contrato'})

# SCORE

In [16]:
print(df['score_desc'].isnull().value_counts())
print(df['score'].isnull().value_counts())

False    186523
Name: score_desc, dtype: int64
False    186523
Name: score, dtype: int64


In [121]:
df['score_desc'].unique()

array(['F-Low Risk', 'A-Very Low Risk', 'C-Very Low Risk',
       'H-Medium Risk', 'No Bureau History Available', 'M-Very High Risk',
       'K-High Risk', 'B-Very Low Risk', 'I-Medium Risk',
       'D-Very Low Risk', 'Not Scored: Only a Guarantor',
       'Not Scored: No Updates available in last 36 months',
       'Not Scored: No Activity seen on the customer (Inactive)',
       'E-Low Risk', 'J-High Risk', 'G-Low Risk',
       'Not Scored: Sufficient History Not Available',
       'Not Scored: Not Enough Info available on the customer',
       'L-Very High Risk',
       'Not Scored: More than 50 active Accounts found'], dtype=object)

In [119]:
# valores de 'score_desc' que não continham a palavra 'Risk' ou 'risk'

other_types = df.loc[(df['score_desc'].str.contains('Risk') == False) |
                    (df['score_desc'].str.contains('Risk') == False),
                    'score_desc'].unique()

print('scores para dados "Not Scored"', df.loc[df['score_desc'].str.contains('No') == True]['score'].unique())

other_types

scores para dados "Not Scored" [ 0 14 18 16 15 17 11]


array(['No Bureau History Available', 'Not Scored: Only a Guarantor',
       'Not Scored: No Updates available in last 36 months',
       'Not Scored: No Activity seen on the customer (Inactive)',
       'Not Scored: Sufficient History Not Available',
       'Not Scored: Not Enough Info available on the customer',
       'Not Scored: More than 50 active Accounts found'], dtype=object)

# justificando eliminar a coluna 'score'(utilizar 'score_desc')

In [120]:
# valores de 'score_desc' que continham palavra 'Risk'
# o que querem dizer as letras?
# são subgrupos de cada tipo de risco

very_low =  df.loc[df['score_desc'].str.contains('Very Low')]
low = df.loc[(df['score_desc'].str.contains('Low')) & (df['score_desc'].str.contains('Very') == False)]
medium =  df.loc[df['score_desc'].str.contains('Medium')]
high = df.loc[(df['score_desc'].str.contains('High')) &(df['score_desc'].str.contains('Very') == False)]
very_high = df.loc[df['score_desc'].str.contains('Very High')]

print(very_low['score_desc'].unique())
print(low['score_desc'].unique())
print(medium['score_desc'].unique())
print(high['score_desc'].unique())
print(very_high['score_desc'].unique())

# Very Low Risk (A, B, C, D)
# Low Risk (E, F, G)
# Medium Risk (H, I)
# High Risk (J, K)
# Very High Risk (L, M)

# o risco aumenta conforme se percorre o alfabeto

['A-Very Low Risk' 'C-Very Low Risk' 'B-Very Low Risk' 'D-Very Low Risk']
['F-Low Risk' 'E-Low Risk' 'G-Low Risk']
['H-Medium Risk' 'I-Medium Risk']
['K-High Risk' 'J-High Risk']
['M-Very High Risk' 'L-Very High Risk']


In [107]:
# como se comporta o 'score', para cada 'score_desc'
# dividir dados pela letra presente no 'score_desc'

df_risk_letter = df.copy()
df_risk_letter = df_risk_letter.loc[(df_risk_letter['score_desc'].str.contains('Risk')) |
                   (df_risk_letter['score_desc'].str.contains('risk'))]
df_risk_letter['risk_letter'] = df_risk_letter['score_desc'].str[0]
df_risk_letter[['risk_letter', 'score']].groupby('risk_letter').mean()

,score
risk_letter,
A,827.646657
B,774.158211
C,741.895311
D,715.966128
E,691.516206
F,666.132175
G,640.968760
H,617.100385
I,586.856651


In [106]:
# dividir dados pelo valor presente no 'score_desc', desconsiderando a letra

df_risk_word = df.copy()
df_risk_word = df_risk_word.loc[(df_risk_word['score_desc'].str.contains('Risk')) |
                 (df_risk_word['score_desc'].str.contains('risk'))]
df_risk_word['risk_word'] = df_risk_word['score_desc'].str[2:]
df_risk_word[['risk_word', 'score']].groupby('risk_word').mean().sort_values('score')

,score
risk_word,
Very High Risk,303.005018
High Risk,474.778707
Medium Risk,603.669353
Low Risk,668.648474
Very Low Risk,765.769443


In [ ]:
# podemos eliminar a coluna 'score' posto que esta é correlacionada à 'score_desc', 
# a coluna 'score_desc' pode ser dividir os dados tanto pelo tipo de risco, quanto pela letra que 
# acompanha a descrição(esta seria mais precisa)

# como tratar dados 'Not Scored'?

# PLOTS

In [9]:
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = (20, 8)

def plot_default_freq(dataframe, series_name, integer = False):
    
    vals = []
    df_aux = dataframe.copy()
    
    if integer == True:    
        df_aux[series_name] = df_aux[series_name].astype(int)
        for val in df_aux[series_name].unique():
            vals.append(val)
        vals.sort()
        df_aux[series_name] = df_aux[series_name].astype(str)
        
    if integer == False:
        for val in df_aux[series_name].unique():
            vals.append(val)
            
    data_list = []
    for value in vals:
        exec("df_{0} = df_aux.loc[df_aux[series_name] == '{0}']".format(value))
        exec("data_list.append(df_{0})".format(value))
    
    pos = 0
    for data in data_list:
        plt.bar([pos], [len(data)], color = 'green')
        plt.bar([pos], [len(data[data['default'] == 1])], color = 'red')
        pos += 1
        
    plt.xticks(range(len(vals)), vals)
    plt.xticks(rotation = 45)
    plt.yticks([])
    plt.show()

In [10]:
# plot_default_freq(df, 'idade', integer = True)

In [11]:
# plot_default_freq(df, 'mes_contrato', integer = False)

In [12]:
# plot_default_freq(df, 'agencia', integer = True)